In [1]:
import pyvista as pv
import torch.profiler
from Kirchhoffpy import lenspy
from Kirchhoffpy import Feedpy
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np
from Kirchhoffpy import coordinate,field_storage
from Kirchhoffpy.Vopy import CO,dotproduct,abs_v
c=299792458
p = pv.Plotter()
srffolder = 'srf/'


In [2]:
# 1. define frequnecy and wavelength
freq = 150*10**9
Lambda = c*1000/freq
k = 2*np.pi/Lambda
print(Lambda,k)


1.9986163866666666 3.1437675329275225


In [3]:
# 2. define coordinate systems
eff_focal_length = 569.56 #mm
coord_ref = coordinate.coord_sys([0,0,0],[0,0,0],axes = 'xyz')

coord_L1 = coordinate.coord_sys([0,0,-(803.9719951339465-4.34990822154231*10)],[np.pi,0,0],axes = 'xyz',ref_coord = coord_ref)
coord_L2 = coordinate.coord_sys([0,0,-(227.64396727901004-4.696706712699847*10)],[np.pi,0,0],axes = 'xyz',ref_coord = coord_ref)
coord_L3 = coordinate.coord_sys([0,0,-(71.77590111674095-2.96556*10)],[np.pi,0,0],axes = 'xyz',ref_coord = coord_ref)

coord_feed_offset = coordinate.coord_sys([90,0,0],[np.pi,0,0],axes = 'xyz',ref_coord = coord_ref)
coord_feed_rotation = coordinate.coord_sys([0,0,0],[0,0,0],axes = 'xyz',ref_coord = coord_feed_offset)
coord_feed = coordinate.coord_sys([0,0,0],[0,0,0],axes = 'xyz',ref_coord = coord_feed_rotation)



coord_sky_ref = coordinate.coord_sys([0,0,0],[np.pi,0,0],axes = 'xyz',ref_coord = coord_ref)
coord_sky = coordinate.coord_sys([0,0,0],[0,-90/569.56,0],axes = 'xyz',ref_coord = coord_sky_ref)


In [4]:
# 3. define input Feedhorn
feed= Feedpy.GaussiBeam(-2.1714724,
                        10.1161095,
                        k,        
                        coord_feed,
                        polarization = 'x')


26584903304305.848


In [5]:
# 4. Lenses and aperture 
#outputfolder = 'output2/150GHz/x90_offaxis_TEST/'
outputfolder = 'output2/150GHz/test/'
L1 = lenspy.simple_Lens(3.36,
                        4.34991*10,# Thickness
                        44.8*10, # diameter
                        srffolder + 'lens1_f2.rsf', 
                        srffolder + 'lens1_f1.rsf',
                        p,
                        coord_L1,
                        name = 'L1',
                        outputfolder = outputfolder)
L2 = lenspy.simple_Lens(3.36,
                        4.69671*10,# Thickness
                        44.8*10, # diameter
                        srffolder + 'lens2_f2.rsf', 
                        srffolder + 'lens2_f1.rsf',
                        p,
                        coord_L2,
                        name = 'L2',
                        outputfolder = outputfolder)

L3 = lenspy.simple_Lens(3.36,
                        2.96556*10,# Thickness
                        44.8*10, # diameter
                        srffolder + 'lens3_f2.rsf', 
                        srffolder + 'lens3_f1.rsf',
                        p,
                        coord_L3,
                        name = 'L3',
                        outputfolder = outputfolder)
#L2 = lenspy.simple_Lens(2,46.9671 ,400, srffolder + 'lens2_f1.rsf', srffolder + 'lens2_f2.rsf',p,[0,0,-227.64396727901004],name = 'L2')
#L3 = lenspy.simple_Lens(2,29.6556 ,400, srffolder + 'lens3_f1.rsf', srffolder + 'lens3_f2.rsf',p,[0,0,-71.77590111674095],name = 'L3')


In [6]:
# 5. fiedl storage
center_grd = field_storage.Spherical_grd(coord_sky,
                                         0,
                                         0,
                                         20,
                                         20,
                                         501,501,
                                         Type = 'ELoverAz', 
                                         far_near = 'far',
                                         distance = 50000)


In [7]:
#N1 =[760,1880]
#N2 = [480,2160]
N1 =[720,2160]
N2 = [350,2240]

N1 =[720,2160]
N2 = [150,124]

torch.set_num_threads(1)
torch._dynamo.config.suppress_errors = True
# Profile the PO_GPU function
with torch.profiler.profile(
   activities=[
         torch.profiler.ProfilerActivity.CPU,
         torch.profiler.ProfilerActivity.CUDA
   ],
   on_trace_ready=torch.profiler.tensorboard_trace_handler('./log'),
   record_shapes=True,
   with_stack=True,use_cuda=True
) as prof:
   # Run the function you want to profile
   torch.cuda.synchronize()
   L3.PO_analysis(#[400,1,400,1],
               [1,N1[0],N1[1],1],
               [1,N2[0],N2[1],1],
               #[5,N2[0],N2[1],1],
            feed,k,
            sampling_type_f1='polar',
            phi_type_f1 = 'less',
            sampling_type_f2='polar',
            phi_type_f2 = 'less',
            po_name = '_po_cur_0mm.h5',
            Method ='POPO')

   # Print profiling results
print('****************')
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))
print(prof.key_averages().table(sort_by="cpu_time_total"))



Gain of the Guassian feedhonr!
-6.631975905905926 dB
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
input power
poynting value max!
0.002770720117815049
13.014576679252004
******
(array([], dtype=int64),)
test vx
1.0000000000000004 0.9999999999999994
cuda:0
cuda:0 cuda:0 cuda:0 cuda:0 cuda:0 cuda:0
Batch size: 69


100%|██████████| 179/179 [00:38<00:00,  4.64it/s]


(array([], dtype=int64),)
test vx
1.0000000000000004 0.9999999999999994
Transform f1
poynting value max!
0.0019451951708986971
8.672619749979134
f2
poynting value max!
0.001421642058775979
8.8747283293015
f2 transmission
poynting value max!
6.382852880099018
****************
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                              aten::mul         0.06%      25.384ms        43.74%       18.398s       4.866ms    

In [8]:
printt


NameError: name 'printt' is not defined

In [ ]:
N1 =[2960 ,2440]
N2 = [620,640]
L2.PO_analysis(#[400,1,400,1],
               [1,N1[0],N1[1],1],
               [1,N2[0],N2[1],1],
               #[5,N2[0],N2[1],1],
            L3,k,
            sampling_type_f1='polar',
            phi_type_f1 = 'less',
            sampling_type_f2='polar',
            phi_type_f2 = 'less',
            po_name = '_po_cur_0mm.h5',
            Method ='POPO')


In [ ]:
N1 =[680,2360]
N2 = [610,305]
L1.PO_analysis(#[400,1,400,1],
               [1,N1[0],N1[1],1],
               [1,N2[0],N2[1],1],
               #[5,N2[0],N2[1],1],
            L2,k,
            sampling_type_f1='polar',
            phi_type_f1 = 'less',
            sampling_type_f2='polar',
            phi_type_f2 = 'less',
            po_name = '_po_cur_0mm.h5',
            Method ='POPO')


In [ ]:
center_grd.grid.x = center_grd.grid.x.ravel()
center_grd.grid.y = center_grd.grid.y.ravel()
center_grd.grid.z = center_grd.grid.z.ravel()
print(center_grd.grid.z.size)


In [ ]:
L1.source(center_grd,
          k,
          far_near = 'far')


In [ ]:
r, theta, phi = center_grd.coord_sys._toSpherical(center_grd.grid.x,center_grd.grid.y,center_grd.grid.z)
co,cx,crho = CO(theta,phi)
E_co = dotproduct(center_grd.E,co)
E_cx = dotproduct(center_grd.E,cx)


In [ ]:
fig = plt.figure(figsize = (7,6))
vmax= np.log10(np.abs(E_co.reshape(501,-1)).max())*20
plt.pcolor(np.log10(np.abs(E_co.reshape(501,-1)))*20,vmax = vmax, vmin = vmax-50)
plt.colorbar()


In [ ]:
fig = plt.figure(figsize = (7,6))
plt.pcolor(np.log10(np.abs(E_cx.reshape(501,-1)))*20,vmax = vmax-40, vmin = vmax-70)
plt.colorbar()


In [ ]:
import h5py as h5
def readh5(fname):
    name = fname.split('.')
    with h5.File(name[0]+'.h5','r') as f:
        co = f['co-polar'][:,:]
        cx = f['cx-polar'][:,:]
        Az = f['Az'][:]
        El = f['El'][:]
    return Az, El, co, cx


In [ ]:
Az, El, grasp_co,grasp_cx = readh5('output2/150GHz/150GHz_test/_H5/_x90_y0GHz.h5')


In [ ]:
fig = plt.figure(figsize = (7,6))
plt.pcolor(np.log10(np.abs(grasp_co.reshape(501,-1)))*20,vmax = vmax-40, vmin = vmax-70)
plt.colorbar()


In [ ]:
fig = plt.figure(figsize = (7,6))
plt.pcolor(np.log10(np.abs(grasp_cx.reshape(501,-1)))*20,vmax = vmax-40, vmin = vmax-70)
plt.colorbar()


In [ ]:
fig  =  plt.figure(figsize= (10,6))
plt.plot(np.log10(np.abs(grasp_co.reshape(501,-1)[250,:])/np.abs(grasp_co.reshape(501,-1)[250,:]).max())*20,'k-')
plt.plot(np.log10(np.abs(E_co.reshape(501,-1)[250,:])/np.abs(E_co.reshape(501,-1)[250,:]).max())*20,'-')


In [ ]:
fig  =  plt.figure(figsize= (10,6))
plt.plot(np.log10(np.abs(grasp_co.reshape(501,-1)[250,:]))*20,'k-')
plt.plot(np.log10(np.abs(E_co.reshape(501,-1)[250,:])**2)*10,'-')


In [ ]:
fig  =  plt.figure(figsize= (10,6))
plt.plot(np.angle(grasp_co.reshape(501,-1)[250,:]/np.abs(grasp_co.reshape(501,-1)[250,:]).max()),'k-')
plt.plot(np.angle(E_co.reshape(501,-1)[250,:]/np.abs(E_co.reshape(501,-1)[250,:]).max()),'-')


In [ ]:
fig  =  plt.figure(figsize= (10,6))
plt.plot(np.log10(np.abs(grasp_co.reshape(501,-1)[250,:]))*20,'k-')
plt.plot(np.log10(np.abs(E_co.reshape(501,-1)[250,:])**2)*10,'-')


In [ ]:
G_co = grasp_co.reshape(501,-1)/grasp_co.reshape(501,-1)[250,250]
H_co = E_co.reshape(501,-1)/E_co.reshape(501,-1)[250,250]
err = H_co-G_co


In [ ]:
fig = plt.figure(figsize=(5,5))
plt.pcolor(np.abs(err))
plt.colorbar()


In [ ]:
fig  =  plt.figure(figsize= (10,6))
plt.plot(Az,np.log10(np.abs(grasp_co.reshape(501,-1)[250,:]))*20,'k-')


In [ ]:
fig  =  plt.figure(figsize= (10,6))
plt.plot(Az,np.log10(np.abs(np.diagonal(grasp_cx.reshape(501,-1))))*20,'k-')
plt.plot(Az,np.log10(np.abs(np.diagonal(center_grd.E.y.reshape(501,-1))))*20,'-')
plt.ylim([0,10])
plt.xlim(-1,1)


In [ ]:
L_lensFp_3   = 7.177590111674096
L_lens3_2    = 15.586806616226909
L_lens2_1    = 57.632802785493645
L_lens1_Lyot = 1.162050628144469
L_Ly_vw      = 22.7114

L_lens1_ref = L_lensFp_3 + L_lens3_2 + L_lens2_1
L_lens2_ref = L_lensFp_3 + L_lens3_2
L_lens3_ref = L_lensFp_3 
L_Ly_ref = L_lens1_ref + L_lens1_Lyot
L_vw_ref = L_Ly_ref + L_Ly_vw


In [ ]:
print(L_lens1_ref*10)
print(L_lens2_ref*10)
print(L_lens3_ref*10)


In [ ]:
Az.size


In [ ]:
import torch

# Define a simple function
def my_function(x):
    return torch.sin(x) + torch.cos(x)

# Convert to TorchScript using tracing
x = torch.randn(100000, 10)
traced_function = torch.jit.trace(my_function,X)



In [ ]:
# Test speed
for i in range(10000):
    traced_function(x)


In [ ]:
for i in range(10000):
    my_function(x)


In [ ]:
import h5py as h5

with h5.File(outputfolder+'L3_po_cur_0mm.h5','r') as f:
    Ex0 = f['f2/Ex'][:]
    Ey0 = f['f2/Ey'][:]
    Ez0 = f['f2/Ez'][:]
    Hx0 = f['f2/Hx'][:]
    Hy0 = f['f2/Hy'][:]
    Hz0 = f['f2/Hz'][:]
    T0 = f['f2/T'][:]
with h5.File(outputfolder+'L3_po_cur_0mm_2.h5','r') as f:
    Ex2 = f['f2/Ex'][:]
    Ey2 = f['f2/Ey'][:]
    Ez2 = f['f2/Ez'][:]
    Hx2 = f['f2/Hx'][:]
    Hy2 = f['f2/Hy'][:]
    Hz2 = f['f2/Hz'][:]
    T2 = f['f2/T'][:]
print(np.abs(Ex0 - Ex2).max())
print(np.abs(Ey0 - Ey2).max())
print(np.abs(Ez0 - Ez2).max())
print(np.abs(Hx0 - Hx2).max())
print(np.abs(Hx0 - Hy2).max())
print(np.abs(Hx0 - Hz2).max())
print(np.abs(T0 - T2).max())


In [ ]:
import torch as T


In [ ]:
a = T.tensor([100,200.0]).to(T.device('cuda'))


In [ ]:
a.mul_(a)


In [ ]:
a


In [ ]:
a = T.tensor([100,200.0]).to(T.device('cuda'))
for i in range(1000000):
    b = a**2


In [ ]:
a = T.tensor([100,200.0]).to(T.device('cuda'))
for i in range(1000000):
    b = a.mul_(a)


In [ ]:
a
